# Playground for the Successive Constraint Method

In this project we construct a reduced basis for solving a linear system $A(\theta) \vec{x}(\theta) = \vec{s}(\theta)$ with a greedy algorithm to improve the snapshot basis. To calibrate the error estimator, which is based on the norm of the residual vector $ \vec{R}(\theta) = A(\theta) X \vec{c}(\theta) - \vec{s}(\theta)$, where $X$ is the snapshot matrix and $\vec{c}$ the coefficient vector corresponding to the reduced basis, we need an estimate for the minimum singular values of $A(\theta)$ in the parameter space $\vec{\theta}$.

Our goal here is to estimate the minimum singular values of a matrix with affine decomposition, which is needed to determine an upper bound for the error estimator in our greedy algorithm. 

This notebook provides a simple yet somewhat realistic scenario for developing and testing the Successive Constraint Method. 

In [ ]:
import numpy as np
from scipy.special import spherical_jn
hbarc = 197.3269718

We are interested in special second-order ordinary differential equations of the following form:
$$
 \frac{d^2 y}{dx^2} = - g(x) y(x) + s(x).
$$

$g(x)$ and $s(x)$ have affine decompositions:

$$
g(x) = g_0(x) + \sum_{i=1}^{n_\theta} \theta_i \, g_i(x)
$$

and likewise for $s(x)$. We concatenate all $n_\theta$ parameters of the system into the vector $\theta$. Hence, $g(x)$ and $s(x)$ have a $\theta$-independent term (with index 0) and a $\theta$-dependent term.

In [ ]:
def potential_eval_affine(r):
    """
    auxiliary function

    the output is an array, where the first column corresponds to the theta-independent contributions, and the following columns to the parameters in the parameter vector theta
    """
    Vcomp = [np.zeros_like(r)]
    r2 = r ** 2
    K_arr = [1.487, 0.465]
    for K in K_arr:
        Vcomp.append(np.exp(-K * r2))
    return np.array(Vcomp).T  / hbarc

def g_s_affine(r, params=None):
    """
    evalutates the affine decompositions of g(x) and s(x) simulaneously 
    
    the output is the tuple (g, s), where both g and s are arrays, where the first column corresponds 
    to the theta-independent contributions, and the following columns to the parameters in the parameter vector theta
    """
    # constants
    l = 0
    MeV = 1./ hbarc
    mu = 938.9182 / 2. * MeV
    E = 50. * MeV
    p = np.sqrt(2.*mu*E)

    # evaluate potential
    V_arr = potential_eval_affine(r)
    g_arr = - (2. * mu) * V_arr
    centrifugal = -l * (l + 1) / r ** 2. if l > 0 else 0.
    g_arr[:, 0] += centrifugal + (2. * mu) * E
    s_arr = np.einsum("i,ij->ij", spherical_jn(l, p*r) * r * (2. * mu), V_arr)
    return g_arr, s_arr

To solve the differential equation above, we use Matrix Numerov's method, which leads to the linear system $A(\theta) \vec{x}(\theta) = \vec{s}(\theta)$. Both, the matrix $A$ and right-hand side vector $\vec{s}$ have affine decompositions. 

In this notebook, we are only interested in the matrix $A$:

$$
A(\theta) = A_0 + \sum_{i=1}^{n_\theta}  A_i(x) \, \theta_i \equiv A_0 + A_\theta \cdot \vec{\theta}
$$

Note that $A_\theta$ is constructed to be a rank-3 tensor. We can obtain the matrices $A_n$ for all $n \geq 0$ using the solver class `AffineNumerovSolver`, which is located in the `modules` folder in this repository:

In [ ]:
from modules.Numerov import AffineNumerovSolver
grid = np.linspace(0, 10, 100)  # grid on which to solve the differential equation
solver = AffineNumerovSolver(xn=grid, g=None, g_s=g_s_affine, y0=0., yp0=0., params=None )
A0, A_theta = solver.A_const_theta_dense

Here, `A0` corresponds to the parameter-independent term $A_0$ and `A_theta` corresponds to the parameter-dependent term $A_\theta$, which has $n_\theta$ components along its third dimension. Hence, we can obtain the full matrix at a given parameter vector $\vec{\theta}^*$ using a simple dot product as follows:

In [ ]:
theta_star = np.array([1., 200., -91.85])   # given parameter vector
A = A0 + A_theta @ theta_star
A

We can perform the (costly) SVD of `A` at any given parameter vector in the snapshot basis. The goal is to estimate the minimal singular values at any point in the parameter space given that snapshot basis and its singular values (and potentially, singular vectors). 

In [ ]:
import matplotlib.pyplot as plt
import scipy.linalg as spla

In [ ]:
num_theta1 = 100
num_theta2 = 100
theta1 = np.linspace(0, 400, num_theta1)
theta2 = np.linspace(-400, 0, num_theta2)
min_sv = np.zeros((num_theta2, num_theta1))
cond = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        theta_vec = np.array([1, t1, t2])
        A_theta_vec = A0 + A_theta @ theta_vec
        s_theta_vec = spla.svdvals(A_theta_vec)
        min_sv[i, j] = s_theta_vec[-1]
        cond[i, j] = s_theta_vec[0] / s_theta_vec[-1]

In [ ]:
fig, ax = plt.subplots()
out = ax.pcolormesh(theta1, theta2, min_sv, shading='gouraud')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Smallest singular value of $A(\theta)$')
_ = fig.colorbar(out)

In [ ]:
fig, ax = plt.subplots()
out = ax.pcolormesh(theta1, theta2, min_sv**2, shading='gouraud')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Smallest eigenvalue of $A(\theta)^T A(\theta)$')
_ = fig.colorbar(out)

In [ ]:
fig, ax = plt.subplots()
out = ax.pcolormesh(theta1, theta2, cond, shading='gouraud')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Condition number of $A(\theta)$')
_ = fig.colorbar(out)

In [ ]:
np.min(min_sv)

In [ ]:
np.max(min_sv)

In [ ]:
A_0 = A0 + A_theta[:, :, 0]
A_1 = A_theta[:, :, 1]
A_2 = A_theta[:, :, 2]

$$
\begin{align*}
(A_0 + \theta_1 A_1 + \theta_2 A_2)^T (A_0 + \theta_1 A_1 + \theta_2 A_2)
= A_0^T A_0 +
\theta_1^2 A_1^T A_1 +
\theta_2^2 A_2^T A_2 +
\theta_1 (A_0^T A_1 + A_1^T A_0) +
\theta_2 (A_0^T A_2 + A_2^T A_0) +
\theta_1 \theta_2 (A_1^T A_2 + A_2^T A_1)
\end{align*}
$$

In [ ]:
np.min(spla.eigvalsh(A.T @ A))

In [ ]:
np.min(spla.eigvalsh(A_0.T @ A_0))

In [ ]:
np.min(spla.eigvalsh(A_1.T @ A_1))

In [ ]:
np.min(spla.eigvalsh(A_2.T @ A_2))

In [ ]:
np.min(spla.eigvalsh(A_0.T @ A_1 + A_1.T @ A_0))

In [ ]:
np.min(spla.eigvalsh(A_0.T @ A_2 + A_2.T @ A_0))

In [ ]:
np.min(spla.eigvalsh(A_1.T @ A_2 + A_2.T @ A_1))

In [ ]:
f0 = lambda theta: 1
f1 = lambda theta: theta[0]**2
f2 = lambda theta: theta[1]**2
f3 = lambda theta: theta[0]
f4 = lambda theta: theta[1]
f5 = lambda theta: theta[0] * theta[1]
fs = (f0, f1, f2, f3, f4, f5)

In [ ]:
M0 = A_0.T @ A_0
M1 = A_1.T @ A_1
M2 = A_2.T @ A_2
M3 = A_0.T @ A_1 + A_1.T @ A_0
M4 = A_0.T @ A_2 + A_2.T @ A_0
M5 = A_1.T @ A_2 + A_2.T @ A_1
Ms = (M0, M1, M2, M3, M4, M5)

In [ ]:
def box_constraints(matrices):
    bc = []
    for M in matrices:
        e = spla.eigvalsh(M)
        bc.append((e[0], e[-1]))
    return tuple(bc)

In [ ]:
bc = box_constraints(Ms)
bc

In [ ]:
lb0 = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        t = (t1, t2)
        lb0[i, j] = np.sum([min(f(t) * m, f(t) * M) for f, (m, M) in zip(fs, bc)]) 

In [ ]:
fig, ax = plt.subplots()
out = ax.pcolormesh(theta1, theta2, lb0, shading='gouraud')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Lower bound for $\sigma_{\min}(A(\theta)^T A(\theta))$')
_ = fig.colorbar(out)

In [ ]:
from scipy.optimize import linprog

In [ ]:
class MinEigBounds():
    def __init__(self, fs, Ms):
        self.fs = fs
        self.Ms = Ms
        self.thetas = []
        self.min_ev = []
        self.ys = []
        self.A_ub = None
        self.bounds = box_constraints(Ms)

    def append(self, theta):
        self.thetas.append(theta)
        mat = sum(f(theta) * M for f, M in zip(self.fs, self.Ms))
        e, X = spla.eigh(mat)
        self.min_ev.append(e[0])
        x = X[:, 0]
        self.ys.append(tuple(M @ x @ x for M in self.Ms))
        A_row = np.array([[-f(theta) for f in self.fs]])
        if self.A_ub is None:
            self.A_ub = A_row
        else:
            self.A_ub = np.vstack((self.A_ub, A_row))

    def lb(self, theta):
        c = np.array([f(theta) for f in self.fs])
        b_ub = -np.array(self.min_ev)
        res = linprog(c, A_ub=self.A_ub, b_ub=b_ub, bounds=self.bounds)
        return res.fun

    def ub(self, theta):
        c = np.array([f(theta) for f in self.fs])
        return min(c @ y for y in self.ys)

In [ ]:
b1 = MinEigBounds(fs, Ms)
b1.append((200, -200))
lb1 = np.zeros((num_theta2, num_theta1))
ub1 = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        lb1[i, j] = b1.lb((t1, t2))
        ub1[i, j] = b1.ub((t1, t2))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

ax = axs[0]
out = ax.pcolormesh(theta1, theta2, lb1, shading='gouraud')
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Lower bound for $\sigma_{\min}(A(\theta)^T A(\theta))$')
_ = fig.colorbar(out)

ax = axs[1]
out = ax.pcolormesh(theta1, theta2, ub1, shading='gouraud')
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Upper bound for $\sigma_{\min}(A(\theta)^T A(\theta))$')
_ = fig.colorbar(out)

In [ ]:
b2 = MinEigBounds(fs, Ms)
thetas2 = [(t1, t2) for t1 in np.linspace(0, 400, 4) for t2 in np.linspace(-400, 0, 4)]
for theta in thetas2:
    b2.append(theta)
lb2 = np.zeros((num_theta2, num_theta1))
ub2 = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        lb2[i, j] = b2.lb((t1, t2))
        ub2[i, j] = b2.ub((t1, t2))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

ax = axs[0]
out = ax.pcolormesh(theta1, theta2, lb2, shading='gouraud')
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Lower bound for $\sigma_{\min}(A(\theta)^T A(\theta))$')
_ = fig.colorbar(out)

ax = axs[1]
out = ax.pcolormesh(theta1, theta2, ub2, shading='gouraud')
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Upper bound for $\sigma_{\min}(A(\theta)^T A(\theta))$')
_ = fig.colorbar(out)

In [ ]:
from time import perf_counter

In [ ]:
def scm(fs, Ms, p_init, p_training_set, tol=1e-1, maxit=100):
    t0 = perf_counter()
    b = MinEigBounds(fs, Ms)
    errors = []
    p_max = p_init
    p_training_set = set(p_training_set)
    for i in range(maxit):
        print(i + 1, end='')
        b.append(p_max)
        p_training_set.discard(p_max)
        if not p_training_set:
            break
        err_max, p_max = max(((1 - b.lb(p)/b.ub(p), p) for p in p_training_set),
                             key=lambda x: x[0])
        print(f', {err_max:.3e}, {p_max}')
        errors.append(err_max)
        if err_max <= tol:
            break
    print('Elapsed time (s):', perf_counter() - t0)
    return b, errors

In [ ]:
training_set = [(x, y) for x in np.linspace(0, 400, 50) for y in np.linspace(-400, 0, 50)]
b_scm, errors_scm = scm(fs, Ms, (200, -200), training_set, tol=0.8, maxit=500)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(*zip(*b_scm.thetas))
ax.set_aspect('equal')
_ = ax.set_title('SCM points')

In [ ]:
it = np.arange(1, len(errors_scm) + 1)
fig, ax = plt.subplots()
ax.loglog(it, errors_scm, '.-', label='error')
ax.loglog(it, 3e2/it, label=r'$O(1/k)$')
ax.set_xlabel('Iteration $k$')
ax.set_title('SCM errors')
ax.grid()
_ = ax.legend()

In [ ]:
lb_scm = np.zeros((num_theta2, num_theta1))
ub_scm = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        lb_scm[i, j] = b_scm.lb((t1, t2))
        ub_scm[i, j] = b_scm.ub((t1, t2))
vmin_scm = np.min(lb_scm)
vmax_scm = np.max(ub_scm)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(11, 4))

ax = axs[0]
out = ax.pcolormesh(theta1, theta2, np.sqrt(lb_scm), shading='gouraud', vmin=np.sqrt(vmin_scm), vmax=np.sqrt(vmax_scm))
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Lower bound for $\sigma_{\min}(A(\theta))$')

ax = axs[1]
out = ax.pcolormesh(theta1, theta2, np.sqrt(ub_scm), shading='gouraud', vmin=np.sqrt(vmin_scm), vmax=np.sqrt(vmax_scm))
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Upper bound for $\sigma_{\min}(A(\theta))$')
_ = fig.colorbar(out)

In [ ]:
class MinSinBounds():
    def __init__(self, cs, As):
        assert len(cs) == len(As)
        self.cs = cs
        self.As = As
        self.fs = []
        self.Ms = []
        for i in range(len(self.cs)):
            for j in range(i, len(self.cs)):
                if i == j:
                    self.fs.append(lambda x, i=i: self.cs[i](x)**2)
                    self.Ms.append(self.As[i].T @ self.As[i])
                else:
                    self.fs.append(lambda x, i=i, j=j: self.cs[i](x) * self.cs[j](x))
                    X = self.As[i].T @ self.As[j]
                    self.Ms.append(X + X.T)
        self.centers = []
        self.samples = {}
        self.min_ev = {}
        self.ys_local = {}
        self.ys = []
        self.min_sv = {}
        self.A_norms = [spla.norm(A) for A in self.As]
        self.bounds = {}

    def _update(self, theta):
        mat = sum(c(theta) * A for c, A in zip(self.cs, self.As))
        U, s, Vh = spla.svd(mat)
        x = Vh[-1]
        self.ys.append(np.array([M @ x @ x for M in self.Ms]))
        self.min_sv[theta] = s[-1]

    def add_center(self, center):
        assert center not in self.centers
        self.centers.append(center)
        self.samples[center] = []
        self.min_ev[center] = []
        self.ys_local[center] = []
        self._update(center)
        self.bounds[center] = tuple((-norm / self.min_sv[center], norm / self.min_sv[center])
                                    for norm in self.A_norms)

    def add_sample(self, center, theta):
        self.samples[center].append(theta)
        self._update(theta)
        mat_c = sum(c(center) * A for c, A in zip(self.cs, self.As))
        mat_t = sum(c(theta) * A for c, A in zip(self.cs, self.As))
        X = spla.solve(mat_c, mat_t)
        X = (X + X.T) / 2
        e, V = spla.eigh(X)
        x = V[:, 0]
        self.min_ev[center].append(e[0])
        y = []
        for A in self.As:
            Y = spla.solve(mat_c, A)
            y.append(Y @ x @ x)
        self.ys_local[center].append(np.array(y))

    def lb_center(self, center, theta):
        c = np.array([c(theta) for c in self.cs])
        A_ub = np.array([[-c(t) for c in self.cs] for t in self.samples[center]])
        b_ub = -np.array(self.min_ev[center])
        res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=self.bounds[center])
        return res.fun

    def ub_center(self, center, theta):
        c = np.array([c(theta) for c in self.cs])
        return min(c @ y for y in self.ys_local[center])

    def lb(self, theta):
        lbs = (self.min_sv[center] * self.lb_center(center, theta)
               for center in self.centers)
        return max(lbs)

    def ub(self, theta):
        c = np.array([f(theta) for f in self.fs])
        return np.sqrt(min(c @ y for y in self.ys))

In [ ]:
c0 = lambda theta: 1
c1 = lambda theta: theta[0]
c2 = lambda theta: theta[1]
cs = (c0, c1, c2)
As = (A_0, A_1, A_2)

In [ ]:
nnb1 = MinSinBounds(cs, As)
nnb1.add_center((200, -200))
nnb1.add_sample((200, -200), (200, -200))
nnlb1 = np.zeros((num_theta2, num_theta1))
nnub1 = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        nnlb1[i, j] = nnb1.lb((t1, t2))
        nnub1[i, j] = nnb1.ub((t1, t2))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

ax = axs[0]
out = ax.pcolormesh(theta1, theta2, nnlb1, shading='gouraud')
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Lower bound for $\sigma_{\min}(A(\theta))$')
_ = fig.colorbar(out)

ax = axs[1]
out = ax.pcolormesh(theta1, theta2, nnub1, shading='gouraud')
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Upper bound for $\sigma_{\min}(A(\theta))$')
_ = fig.colorbar(out)

In [ ]:
def cnnscm(cs, As, p_init, p_training_set, tol_local=0.8, tol_global=0.8):
    t0 = perf_counter()
    b = MinSinBounds(cs, As)
    center = p_init
    p_training_set = set(p_training_set)
    positivity_remainder = p_training_set.copy()
    i = 0
    while True:
        i += 1
        print(f'Center {i}: {center}')
        b.add_center(center)
        p_max = center
        R_old = set()
        R_new = set()
        j = 0
        while True:
            j += 1
            print(f'    {j}', end='')
            b.add_sample(center, p_max)
            R_old = R_new
            R_new = set(p for p in p_training_set if b.lb_center(center, p) > 0)
            if len(R_new) == len(p_training_set):
                print(': Global positivity')
                break
            err_max, p_max = max(((1 - b.lb_center(center, p)/b.ub_center(center, p), p) for p in p_training_set),
                                 key=lambda x: x[0])
            R_diff = R_new - R_old
            print(f', {len(R_diff):4d}, {err_max:.3e}, {p_max}')
            if not R_diff and err_max <= tol_local:
                print('    No new positivity and error below tolerance')
                break
        positivity_remainder -= R_old
        if not positivity_remainder:
            print('Global positivity achieved')
            err_max, p_max = max(((1 - b.lb(p)/b.ub(p), p) for p in p_training_set),
                                 key=lambda x: x[0])
            print(f'{err_max:.3e}, {p_max}')
            if err_max <= tol_global:
                print('Global error below tolerance')
                break
            center = p_max
        else:
            print(f'Negativity: {len(positivity_remainder)}')
            _, center = min(((b.lb_center(center, p), p) for p in positivity_remainder),
                            key=lambda x: x[0])
    print('Elapsed time (s):', perf_counter() - t0)
    return b

In [ ]:
b_cnnscm = cnnscm(cs, As, (200, -200), training_set)

In [ ]:
len(b_cnnscm.centers)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(*zip(*b_cnnscm.centers))
ax.set_aspect('equal')
_ = ax.set_title('CNNSCM centers')

In [ ]:
sum(len(x) for x in b_cnnscm.samples.values())

In [ ]:
fig, ax = plt.subplots()
for x in b_cnnscm.samples.values():
    ax.scatter(*zip(*x))
ax.set_aspect('equal')
_ = ax.set_title('CNNSCM samples')

In [ ]:
i = 0
c = b_cnnscm.centers[i]
fig, ax = plt.subplots()
ax.scatter(*zip(*b_cnnscm.samples[c]))
ax.scatter(*c, marker='x')
ax.set_aspect('equal')
ax.set_xlim(0, 400)
ax.set_ylim(-400, 0)
_ = ax.set_title('CNNSCM samples')

In [ ]:
lb_cnnscm = np.zeros((num_theta2, num_theta1))
ub_cnnscm = np.zeros((num_theta2, num_theta1))
for j, t1 in enumerate(theta1):
    for i, t2 in enumerate(theta2):
        lb_cnnscm[i, j] = b_cnnscm.lb((t1, t2))
        ub_cnnscm[i, j] = b_cnnscm.ub((t1, t2))
vmin_cnnscm = np.min(lb_cnnscm)
vmax_cnnscm = np.max(ub_cnnscm)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(11, 4))

ax = axs[0]
out = ax.pcolormesh(theta1, theta2, lb_cnnscm, shading='gouraud', vmin=vmin_cnnscm, vmax=vmax_cnnscm)
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Lower bound for $\sigma_{\min}(A(\theta))$')

ax = axs[1]
out = ax.pcolormesh(theta1, theta2, ub_cnnscm, shading='gouraud', vmin=vmin_cnnscm, vmax=vmax_cnnscm)
ax.set_aspect('equal')
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.set_title(r'Upper bound for $\sigma_{\min}(A(\theta))$')
_ = fig.colorbar(out)